# Some specific applications


## 1. See whether an item is available in a store
For some aaplications you might need to verify if an item exists in a store without using a `get` expression. Internally the store's items are saved as a set calles `store.items`. If you want to check the pressence of an item in a store, you can simply use `item in store.items`. You can also manipulate items in the store directly this way, but you will bypass the scheduling, so be careful when doing this.

Below you have a small example:

In [4]:
using SimJulia

In [17]:
# checking for presence
mutable struct mytype
    id::Int
    status::Int
end

@resumable function putter(sim::Simulation,s::Store, m::mytype)
    @yield timeout(sim,10)
    put(s, m)
    @yield timeout(sim, 2)
    get(s)
end

@resumable function checker(sim::Simulation, s::Store, m::mytype)
    while true
        @yield timeout(sim,1)
        if m in s.items
            println("found the item on time $(now(sim))")
        end
    end
end
   
sim = Simulation()
store = Store{mytype}(sim)
m = mytype(12,0)
@process putter(sim,store, m)
@process checker(sim, store, m)
run(sim,15)

found the item on time 10.0
found the item on time 11.0


In [53]:
# checking for presence based on property
mutable struct mytype
    id::Int
    status::Int
end

@resumable function putter(sim::Simulation,s::Store)
    i = 1
    while true
        @yield timeout(sim,1)
        put(s, mytype(i,rand([0,1])))
        i += 1
    end
end

@resumable function checker(sim::Simulation, s::Store, n::Int)
    while true
        @yield timeout(sim,1)
        if n in [x.id for x in s.items]
            println("found the item $(n) on time $(now(sim))")
        end
    end
end
   
sim = Simulation()
store = Store{mytype}(sim)
# modify the store without the scheduler:
push!(store.items, mytype(172,172))
# ----------------------------------
@process putter(sim,store)
@process checker(sim, store, 9)
run(sim,15)

found the item 9 on time 9.0
found the item 9 on time 10.0
found the item 9 on time 11.0
found the item 9 on time 12.0
found the item 9 on time 13.0
found the item 9 on time 14.0


In [55]:
println(store.items)
# Modify the store without the schuduler:
store.items = filter!(x -> x.id  ∉ [172, 10, 11, 12, 13, 14], store.items)

Set(mytype[mytype(12, 1), mytype(2, 0), mytype(9, 0), mytype(13, 1), mytype(10, 0), mytype(4, 0), mytype(7, 1), mytype(14, 1), mytype(5, 0), mytype(1, 1), mytype(172, 172), mytype(11, 0), mytype(3, 0), mytype(6, 1), mytype(8, 0)])


Set(mytype[mytype(2, 0), mytype(9, 0), mytype(4, 0), mytype(7, 1), mytype(5, 0), mytype(1, 1), mytype(3, 0), mytype(6, 1), mytype(8, 0)])

## 2. Generate a model based on historical data
Suppose you have some historical data in a tabular form (Excel/CSV file) that you want to process and use to fit a model for a simulation.

In [193]:
using DataFrames, Query, XLSX # for data manipulation and import
using Dates                   # for datetime functionality
# read the files
df = DataFrame(XLSX.readtable("demofile.xlsx","Sheet1")...)
# set column type to dates
df.date = Dates.Date.(df.date)
# show DataFrame
df

,Asset,date
,Any,Date
1,CH001,2015-03-01
2,CH002,2015-01-02
3,CH003,2015-04-23
4,CH002,2016-12-19
5,CH001,2014-05-08
6,CH002,2016-05-23
7,CH003,2014-11-06
8,CH002,2014-07-09
9,CH001,2017-03-03


In [197]:
# extract time between events
TBF = []
for asset in Set(df.Asset)
    # make intermediate dataframe for specific asset
    tdf =  @from i in df begin  
            @where i.Asset == asset   # filter on asset name
            @select {i.date}          # select dates only
            @collect DataFrame        # return dataframe     
    end
    # get sorted dates for specific asset
    mdates = sort(tdf.date) 
    # calculate time between events
    res = mdates[2:end] - mdates[1:end-1] 
    # store result
    append!(TBF,res)
end
TBF

17-element Array{Any,1}:
 297 days 
 42 days  
 110 days 
 145 days 
 69 days  
 367 days 
 149 days 
 696 days 
 168 days 
 511 days 
 1017 days
 52 days  
 177 days 
 101 days 
 406 days 
 151 days 
 59 days  

In [198]:
# Make distribution
using Distributions, HypothesisTests
# try poisson model
mydist = fit(Exponential, Dates.value.(TBF))
# evaluate the fit
ExactOneSampleKSTest(Dates.value.(TBF), mydist)

Exact one sample Kolmogorov-Smirnov test
----------------------------------------
Population details:
    parameter of interest:   Supremum of CDF differences
    value under h_0:         0.0
    point estimate:          0.1607412254373159

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.7138

Details:
    number of observations:   17
